Production (Importing, Functions, Team Spelling, Merging, Exporting)

In [289]:
# Import Libraries and Data
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 300)
pd.options.mode.chained_assignment = None

# MDCM
mdcm = pd.read_csv('../data/mdcm/NCAA_Tourney_2002_2023.csv')
team_spellings = pd.read_csv('../data/mdcm/team_spellings.csv')

# CBBData
selection_sunday_resume = pd.read_csv('../data/cbbdata/team/selection_sunday_resume.csv')

# Kaggle
barttovik_home = pd.read_csv('../data/kaggle/march_madness_data/barttovik_home.csv')
barttovik_away = pd.read_csv('../data/kaggle/march_madness_data/barttovik_away.csv')
kenpom_barttovik = pd.read_csv('../data/kaggle/march_madness_data/kenpom_barttovik.csv')
shooting_splits = pd.read_csv('../data/kaggle/march_madness_data/shooting_splits.csv')

In [294]:
# Define Functions
def merge_team_season(df: pd.DataFrame, df_merge_onto: pd.DataFrame, filter_df_merge_onto_year = None, title: str = None):
    """
    
        Function to merge teams and their seasons in college basketball with a bevy of alternative spellings, using team_spellings.csv.
    
        df (pd.DataFrame): The dataframe you'd like to establish as your left, or original df. Must contain 'teamname' and 'season' columns.
        
        df_merge_onto (pd.DataFrame): The dataframe you'd like to left merge onto df. Must contain 'team' and 'year' columns.
        
        filter_df_merge_onto_year (int): If there is a cutoff year for the df_merge_onto, the year of interest. This does not affect the 
                                        merge, just the reporting success messages.
    
    """
    # Print Title of Run For Terminal
    if title:
        print(title,'-----------------------\n')
        
    # If Oncoming Data Filtered By Specifc Year 
    if filter_df_merge_onto_year != None:
        df_post_cutoff = df[df['season'] >= filter_df_merge_onto_year]
        df_pre_cutoff = df[df['season'] < filter_df_merge_onto_year]
        df = df_post_cutoff
    
    # Find Null df Column Identifier For Oncoming DF
    for col_name in df_merge_onto.columns:
        if 'team' not in col_name.lower() and 'year' not in col_name.lower():
            col_null_match_identifier = col_name
            print('Column Null Match Identifier:', col_null_match_identifier,'\n')
            break
     
    # Set Both Teamname Columns To Lowercase Easier Merging
    for team_num in range(1,3):
        df[f'team{team_num}_teamname'] = df[f'team{team_num}_teamname'].str.lower()
    
    # Loop Through Team 1 and Team 2
    df_both_teams = pd.DataFrame()
    for team_num in range(1, 3):
        
        # Adjust Column Names Due To Team1 and Team2 (Remove 'team1_' or 'team1_'/'team2_')
        if team_num == 2:
            df_merge_onto.columns = df_merge_onto.columns.str[6:]
        df_merge_onto = df_merge_onto.add_prefix(f'team{team_num}_')
        
        # Establish df_merge_onto Team Column Name and Set to Lowercase
        df_merge_onto[f'team{team_num}_team'] = df_merge_onto[f'team{team_num}_team'].str.lower()
        
        # If Second Iteration, Find The Merge Columns and The Spellings Ones To Keep
        if team_num == 2:
            spellings = ['team2_teamname', 'season']
            for col in df.columns:
                if f"team{team_num}_name_spelling" in col: 
                    spellings.append(col)
            df = df[spellings].drop_duplicates()
            
        # Original Merge For Team 1 or 2
        print(f"Team {team_num} Merge ...\n")
        df_merged = pd.merge(df, df_merge_onto, how = 'left', left_on = [f'team{team_num}_teamname', 'season'], right_on = [f'team{team_num}_team', f'team{team_num}_year'])
        
        # Original Split Up Merged and Unmerged Data
        df_not_merged = df_merged[df_merged[f'team{team_num}_{col_null_match_identifier}'].isna() == True]
        df_merged = df_merged[df_merged[f'team{team_num}_{col_null_match_identifier}'].isna() == False]
        
        print(f'Original Team {team_num} Merge:', len(df), 'total rows.')
        print('Matched During Iteration:', len(df_merged)) 
        print('Unmatched Rows Remaining:', len(df_not_merged), '\n') 
        
        # Remove Columns That Didn't Merge Properly Based On Num of Columns
        # Reduce To Team and Year, Along with Alternate Spellings
        neg_col_count_df_merge_onto = df_merge_onto.shape[1] * -1
        df_not_merged = df_not_merged.iloc[:, :neg_col_count_df_merge_onto]
        
        # Loop Through Columnns To Fix The Merge
        print(f"Correcting Team {team_num} Merge ...\n")
        merge_complete, i = False, 1
        while merge_complete == False:
            
            # Perform Loop Everytime More Unmatched Columns Are Found
            team_season_loop = pd.merge(df_not_merged, df_merge_onto, how = 'left', left_on = [f'team{team_num}_name_spelling_{i}', 'season'], right_on = [f'team{team_num}_team', f'team{team_num}_year'])
            print(f'Team {team_num} Season Loop {i}:', len(team_season_loop), 'total rows.')
            
            # Split Up The Matched and Unmatched
            matched_df = team_season_loop[team_season_loop[f'team{team_num}_{col_null_match_identifier}'].isna() == False]
            print('Matched During Iteration:', len(matched_df)) 
            unmatched_df = team_season_loop[team_season_loop[f'team{team_num}_{col_null_match_identifier}'].isna() == True]
            print('Unmatched Rows Remaining:', len(unmatched_df), '\n') 
            
            # For The DataFrames With Data In The Column From Second DF, Add To team_season
            if len(matched_df) > 0:
                df_merged = pd.concat([df_merged, matched_df])
            # If There Are Still Null Rows, Throw Those Back In The Loop For The Next Iteration
            if len(unmatched_df) > 0:
                df_not_merged = unmatched_df.iloc[:, :neg_col_count_df_merge_onto]
            # If There Aren't Any Null Rows Left, End The Loop
            else:
                print(f'Success! Team {team_num} Merge Completed Early!\n')
                merge_complete = True
            if i == 11:
                merge_complete = True 
            i += 1
   
        # Concat Team 1 or 2 Onto Full DataFrame
        if team_num == 2:
            spellings = []
            for col in df_merged.columns:
                if 'team2_name_spelling' not in col:
                    spellings.append(col)

            df_both_teams = pd.merge(df_both_teams, df_merged[spellings], how = 'left', on = [f'team2_teamname', 'season'])
        else:
            if len(df_not_merged) > 0:
                df_both_teams = pd.concat([df_merged, df_not_merged])
    
    # Inspect Final Results
    print("Filter Views of Resulting DataFrame -------------------------\n")
    for team_num in range(1, 3):
        print(f'Team {team_num}:')
        print('Null Match Identifier Column:', col_null_match_identifier)
        if filter_df_merge_onto_year != None:
            
            # Gather Data About Merge Post Cutoff
            post_cutoff_rows = df_both_teams[(df_both_teams[f'team{team_num}_{col_null_match_identifier}'].isna() == False) & (df_both_teams['season'] >= filter_df_merge_onto_year)]
            post_cutoff_rows_na = df_both_teams[(df_both_teams[f'team{team_num}_{col_null_match_identifier}'].isna() == True) & (df_both_teams['season'] >= filter_df_merge_onto_year)]
            
            if post_cutoff_rows_na.shape[0] > 0:
                print(f'Oh No! There were {post_cutoff_rows.shape[0]} matches and {post_cutoff_rows_na.shape[0]} non matches post {filter_df_merge_onto_year}.')
                unique_unmatched_teams = post_cutoff_rows_na[f'team{team_num}_teamname'].drop_duplicates().sort_values()
                print(f'\n{len(unique_unmatched_teams)} team name(s) from the Team {team_num} Merge that exist(s) in the original df:\n')
                print(unique_unmatched_teams,'\n')
            else:
                print(f'Great! No Null Rows Post {filter_df_merge_onto_year}')
     
            # Gather Data About Merge Pre Cutoff
            pre_cutoff_rows = df_both_teams[(df_both_teams[f'team{team_num}_{col_null_match_identifier}'].isna() == False) & (df_both_teams['season'] < filter_df_merge_onto_year)].shape[0]
            if pre_cutoff_rows == 0:
                print(f'Great! No Matched Rows Pre {filter_df_merge_onto_year}\n')
                                
    # Concat Pre and Post Cutoff If Exists
    if filter_df_merge_onto_year != None:
        df_both_teams = pd.concat([df_both_teams, df_pre_cutoff], ignore_index=True)    
    
    # Fix Rare Situation Where There Is A Duplicate of Team1_Teamname, Team2_Teamname, and Season (If Something To Be Dropped, Then Drop It)
    df_both_teams.drop_duplicates(subset=['team1_teamname', 'team2_teamname', 'season'], keep = 'first', inplace = True)
    
    return df_both_teams.drop(['team1_team','team1_year', 'team2_team', 'team2_year'], axis = 1)

# Ammend List To Remove Duplicate Columns And Retain One Set 
def drop_dup_columns(df: pd.DataFrame, dup_cols_keep: list[str]):
    same_data_columns = []
    for i in range(df.shape[1] - 1):
        for j in range(i + 1, df.shape[1]):
            col1, col2 = df.columns[i], df.columns[j]
            if df[col1].equals(df[col2]):
                same_data_columns.append((col1, col2))
    if not same_data_columns:
        print("No columns have the same data.")
    else:
        for col_pair in same_data_columns:
            if col_pair not in dup_cols_keep:
                df.drop(columns = col_pair[0], axis = 1, inplace = True)
                df.rename(columns = {col_pair[1]: col_pair[1].replace('_away', '')}, inplace = True)
    return df

In [291]:
# Create Round Column
mdcm['round'] = mdcm['slot'].str.extract(r'(\d+)')
mdcm['round'] = pd.to_numeric(mdcm['round'], errors='coerce')

# Adjust Team Spellings
team_spellings = team_spellings.pivot_table(index='team_id', columns=team_spellings.groupby('team_id').cumcount(), values='name_spelling', aggfunc='first')
team_spellings.columns = [f'name_spelling_{i + 1}' for i in range(team_spellings.shape[1])]
team_spellings.reset_index(inplace=True)

# Merge Team Spellings
team_spellings_t1 = team_spellings.add_prefix('team1_')
mdcm = pd.merge(mdcm, team_spellings_t1, how = 'inner', left_on = ['team1_id'], right_on = ['team1_team_id'])
team_spellings_t2 = team_spellings.add_prefix('team2_')
mdcm = pd.merge(mdcm, team_spellings_t2, how = 'inner', left_on = ['team2_id'], right_on = ['team2_team_id'])
mdcm.head(1)

# Reduce MDCM To Columns of Interest
# Simple Seed Data, Geographies, Team & Coach Counting Tourney Stats, and Preseason Rankings
mdcm.drop(columns = ['num_ot', 'WLoc', 'strongseed', 'team1_position', 'team2_position', 'weakseed', 'team1_region', 'team2_region', 'host',
 'host_lat', 'host_long', 'team1_lat', 'team1_long', 'team2_lat', 'team2_long',  'team1_pt_school_ncaa', 'team1_pt_overall_ncaa','team1_pt_school_s16',
 'team1_pt_overall_s16','team1_pt_school_ff','team1_pt_overall_ff','team1_pt_career_school_wins','team1_pt_career_school_losses','team1_pt_career_overall_wins',
 'team1_pt_career_overall_losses','team1_pt_team_season_wins','team1_pt_team_season_losses','team1_pt_coach_season_wins','team1_pt_coach_season_losses',
 'team2_coach_id','team2_pt_school_ncaa','team2_pt_overall_ncaa','team2_pt_school_s16','team2_pt_overall_s16','team2_pt_school_ff','team2_pt_overall_ff',
 'team2_pt_career_school_wins', 'team2_pt_career_school_losses','team2_pt_career_overall_wins','team2_pt_career_overall_losses','team2_pt_team_season_wins',
 'team2_pt_team_season_losses','team2_pt_coach_season_wins','team2_pt_coach_season_losses','team1_ap_final','team1_ap_preseason','team1_coaches_before_final',
 'team1_coaches_preseason','team2_ap_final','team2_ap_preseason','team2_coaches_before_final','team2_coaches_preseason'], inplace= True)

In [292]:
# Merge MDCM and Selection Sunday (2008-2023)
selection_sunday_resume = selection_sunday_resume[selection_sunday_resume['year'] != 'Year']
selection_sunday_resume['year'] = selection_sunday_resume['year'].astype(int)
selection_sunday_resume.drop(columns = ['seed'], axis = 1, inplace = True)
selection_sunday_resume.rename(columns={'score': 'team_score'}, inplace=True)
df = merge_team_season(mdcm, selection_sunday_resume, filter_df_merge_onto_year = 2008, title = 'MDCM and Selection Sunday (2008 - 2023)')

# Merge MDCM and Kenpom Barttovik (2008-2023)
kenpom_barttovik.columns = kenpom_barttovik.columns.str.lower()
kenpom_barttovik = kenpom_barttovik[['year', 'team', 'barthag', 'avg hgt', 'eff hgt', 'exp', 'talent', 'elite sos']]
df = merge_team_season(df, kenpom_barttovik, filter_df_merge_onto_year = 2008, title = 'MDCM and Kenpom Barttovik (2008 - 2023)')

# Merge Home Team Barttovik Data (2008-2023)
barttovik_home.columns = barttovik_home.columns.str.lower()
barttovik_home.drop(columns = ['seed', 'round', 'wab'], axis = 1, inplace = True)
barttovik_home.rename(columns={col: f'home_' + col for col in barttovik_home.columns[-1 * (barttovik_home.shape[1]):]}, inplace=True)
barttovik_home = barttovik_home[['home_year','home_team', 'home_badj em', 'home_badj o', 'home_badj d', 'home_barthag', 
                                 'home_badj t','home_3pt%', 'home_ft%']]
barttovik_home.rename(columns={f'home_team':'team', 'home_year':'year'}, inplace = True)
df = merge_team_season(df, barttovik_home, filter_df_merge_onto_year = 2008, title = 'MDCM and Home Barttovic Data (2008 - 2023)')

# Merge Away Team Barttovik Data (2008-2023)
barttovik_away.columns = barttovik_away.columns.str.lower()
barttovik_away.drop(columns = ['seed', 'round', 'wab'], axis = 1, inplace = True)
barttovik_away.rename(columns={col: f'away_' + col for col in barttovik_away.columns[-1 * (barttovik_away.shape[1]):]}, inplace=True)
barttovik_away = barttovik_away[['away_year','away_team', 'away_badj em', 'away_badj o', 'away_badj d', 'away_barthag', 
                                 'away_badj t','away_3pt%', 'away_ft%']]
barttovik_away.rename(columns={f'away_team':'team', 'away_year':'year'}, inplace = True)
df = merge_team_season(df, barttovik_away, filter_df_merge_onto_year = 2008, title = 'MDCM and Away Barttovic Data (2008 - 2023)')

# Shooting Splits (2010-2023)
shooting_splits.columns = shooting_splits.columns.str.lower()
shooting_splits = shooting_splits.drop(['conf'], axis=1, errors='ignore')
df = merge_team_season(df, shooting_splits, filter_df_merge_onto_year = 2010, title = 'MDCM and Kenpom Barttovik (2013 - 2023)')

# Remove Rank and Alternate Spellingsa From DataFrame and Export
df = df.loc[:, ~df.columns.str.contains('rank|name_spelling', case=False)].dropna(axis=1, how='all')
df.to_csv("../data/pipeline/merged_team_season.csv", index = False)

print(df.shape)
df.head()

MDCM and Selection Sunday (2008 - 2023) -----------------------

Column Null Match Identifier: net 

Team 1 Merge ...

Original Team 1 Merge: 996 total rows.
Matched During Iteration: 782
Unmatched Rows Remaining: 214 

Correcting Team 1 Merge ...

Team 1 Season Loop 1: 214 total rows.
Matched During Iteration: 0
Unmatched Rows Remaining: 214 

Team 1 Season Loop 2: 214 total rows.
Matched During Iteration: 9
Unmatched Rows Remaining: 205 

Team 1 Season Loop 3: 205 total rows.
Matched During Iteration: 101
Unmatched Rows Remaining: 104 

Team 1 Season Loop 4: 104 total rows.
Matched During Iteration: 37
Unmatched Rows Remaining: 67 

Team 1 Season Loop 5: 67 total rows.
Matched During Iteration: 6
Unmatched Rows Remaining: 61 

Team 1 Season Loop 6: 61 total rows.
Matched During Iteration: 2
Unmatched Rows Remaining: 59 

Team 1 Season Loop 7: 59 total rows.
Matched During Iteration: 0
Unmatched Rows Remaining: 59 

Team 1 Season Loop 8: 59 total rows.
Matched During Iteration: 1
Unma

,team1_id,team1_score,team2_id,team2_score,team1_seed,team2_seed,slot,team1_teamname,team2_teamname,season,team1_coach_id,team1_fg2pct,team1_fg3pct,team1_ftpct,team1_blockpct,team1_oppfg2pct,team1_oppfg3pct,team1_oppftpct,team1_oppblockpct,team1_f3grate,team1_oppf3grate,team1_arate,team1_opparate,team1_stlrate,team1_oppstlrate,team2_fg2pct,team2_fg3pct,team2_ftpct,team2_blockpct,team2_oppfg2pct,team2_oppfg3pct,team2_oppftpct,team2_oppblockpct,team2_f3grate,team2_oppf3grate,team2_arate,team2_opparate,team2_stlrate,team2_oppstlrate,team1_tempo,team1_adjtempo,team1_oe,team1_adjoe,team1_de,team1_adjde,team2_tempo,team2_adjtempo,team2_oe,team2_adjoe,team2_de,team2_adjde,game_id,round,team1_team_id,team2_team_id,team1_net,team1_resume,team1_wab,team1_elo,team1_power,team1_q1_w,team1_q2_w,team1_q3q4_l,team1_games_above_500,team1_team_score,team1_type,team1_rd,team2_net,team2_resume,team2_wab,team2_elo,team2_power,team2_q1_w,team2_q2_w,team2_q3q4_l,team2_games_above_500,team2_team_score,team2_type,team2_rd,team1_barthag,team1_avg hgt,team1_eff hgt,team1_exp,team1_talent,team1_elite sos,team2_barthag,team2_avg hgt,team2_eff hgt,team2_exp,team2_talent,team2_elite sos,team1_home_badj em,team1_home_badj o,team1_home_badj d,team1_home_barthag,team1_home_badj t,team1_home_3pt%,team1_home_ft%,team2_home_badj em,team2_home_badj o,team2_home_badj d,team2_home_barthag,team2_home_badj t,team2_home_3pt%,team2_home_ft%,team1_away_badj em,team1_away_badj o,team1_away_badj d,team1_away_barthag,team1_away_badj t,team1_away_3pt%,team1_away_ft%,team2_away_badj em,team2_away_badj o,team2_away_badj d,team2_away_barthag,team2_away_badj t,team2_away_3pt%,team2_away_ft%,team1_team no,team1_team id,team1_dunks fg%,team1_dunks share,team1_dunks fg%d,team1_dunks d share,team1_close twos fg%,team1_close twos share,team1_close twos fg%d,team1_close twos d share,team1_farther twos fg%,team1_farther twos share,team1_farther twos fg%d,team1_farther twos d share,team1_threes fg%,team1_threes share,team1_threes fg%d,team1_threes d share,team2_team no,team2_team id,team2_dunks fg%,team2_dunks share,team2_dunks fg%d,team2_dunks d share,team2_close twos fg%,team2_close twos share,team2_close twos fg%d,team2_close twos d share,team2_farther twos fg%,team2_farther twos share,team2_farther twos fg%d,team2_farther twos d share,team2_threes fg%,team2_threes share,team2_threes fg%d,team2_threes d share
0,1314,81,1181,77,8,2,R5WX,north carolina,duke,2022,hubert-davis-1,50.714286,36.206897,77.219430,9.814529,48.299845,34.920635,73.696145,8.492063,37.437934,36.878049,53.618421,48.031496,0.076692,0.093829,55.844156,36.821192,72.881356,13.258110,46.897038,31.396957,67.160494,8.097785,36.579457,33.769267,56.987116,51.793722,0.094448,0.091014,70.2181,70.1745,109.4160,113.0350,100.9740,96.9911,68.2765,67.4185,117.1520,119.3570,98.2346,95.6444,2022-1314-1181,5,1314,1181,31,48,19,24,26.7,3,6,1,15.0,95.5,At-Large,Finals,12,20,7,9,11.3,6,12,1,22.0,99.7,At-Large,F4,0.866,78.254,81.474,1.742,87.453,29.665,0.943,78.372,81.821,0.988,91.521,28.364,18.3,113.0,94.7,0.884,70.2,39.1,74.5,21.4,118.1,96.7,0.909,70.0,38.1,76.5,17.7,114.7,97.0,0.873,72.0,35.4,79.8,34.1,123.9,89.8,0.976,66.1,39.3,68.8,904.0,135.0,86.8,8.6,88.3,4.9,61.5,35.6,59.0,32.4,34.6,26.4,35.7,31.4,35.8,38.0,33.7,36.2,927.0,50.0,89.7,13.4,86.2,7.5,66.7,38.7,55.4,36.5,39.8,25.6,35.9,29.9,36.6,35.7,32.0,33.6
1,1242,85,1181,81,1,2,R4X1,kansas,duke,2018,bill-self-1,56.536220,40.256709,70.000000,10.973725,48.145286,32.670807,71.976967,10.074938,41.642371,38.351596,56.445312,48.645598,0.094340,0.080084,55.648218,37.793103,70.827489,13.189270,44.783905,32.092426,73.875803,6.747536,35.469667,36.727959,57.341270,55.699177,0.105386,0.081589,70.1000,68.4000,120.9000,116.0000,97.7000,101.4000,71.0000,70.0000,122.6000,118.7000,93.5000,97.3000,2018-1242-1181,4,1242,1181,5,3,4,7,7.7,12,21,1,20.0,99.9,Auto,F4,6,6,9,12,3,6,13,0,19.0,99.9,At-Large,E8,0.941,77.704,80.921,1.783,80.440,34.024,0.958,79.097,82.429,0.688,92.943,31.360,25.6,121.9,96.3,0.

Testing

In [272]:
# Coach Results
coach_results = pd.read_csv('../data/kaggle/march_madness_data/coach_results.csv')
coach_results.columns = coach_results.columns.str.lower()
# coach_results.sort_values(by = 'pake', ascending = False).head()

coach_results['games'].value_counts()

1     99
2     61
3     33
4     19
5     15
8     12
19     7
9      7
6      7
12     6
17     5
7      5
11     4
16     3
25     2
15     2
24     2
44     2
29     2
13     2
14     2
20     2
40     1
31     1
18     1
28     1
51     1
22     1
47     1
10     1
26     1
53     1
32     1
30     1
45     1
Name: games, dtype: int64

In [293]:
# Upset  Counts (2023-2008)
upset_counts = pd.read_csv('../data/kaggle/march_madness_data/upset_counts.csv').drop(columns = ['TOTAL'])
upset_counts.columns = upset_counts.columns.str.lower()
# upset_counts = upset_counts[upset_counts['year'] != 2023]

# Round Counts (2023-2008)
mdcm_non_nit = mdcm[(mdcm['round'] <= 6) & (mdcm['season'] >= 2008)] #(mdcm['season'] != 2023) 
mdcm_non_nit_games = pd.DataFrame(mdcm_non_nit['round'].value_counts())

for i in range(len(upset_counts.columns[1:])):
    upset_counts[f'{upset_counts.columns[i + 1]}_upset'] = round(upset_counts[f'{upset_counts.columns[i + 1]}']/(mdcm_non_nit_games.iloc[i,0]/upset_counts.shape[0]),2)

upset_counts.mean()

year                  2015.200000
first round              7.133333
second round             3.333333
sweet 16                 1.933333
elite 8                  1.200000
final 4                  0.200000
first round_upset        0.224000
second round_upset       0.207333
sweet 16_upset           0.242000
elite 8_upset            0.300000
final 4_upset            0.100000
dtype: float64

In [150]:
# Game Factors - Explore Team Momentum Going Into Tourney
game_factors = pd.read_csv('../data/cbbdata/game/game_factors.csv').query('year != 2020')
game_factors['date']= pd.to_datetime(game_factors['date'])
game_factors[game_factors['type'] != 'post'].head()

C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\3447347392.py:5: DtypeWarning: Columns (25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  game_factors = pd.read_csv('../data/cbbdata/game/game_factors.csv').query('year != 2020')


,date,type,team,conf,opp,opp_conf,loc,result,pts_scored,pts_allowed,adj_o,adj_d,off_ppp,off_efg,off_to,off_or,off_ftr,def_ppp,def_efg,def_to,def_or,def_ftr,game_score,season,tempo,game_id,coach,opp_coach,year,location,avg_marg
224,2008-03-16,conf_t,Arkansas,SEC,Georgia,SEC,N,L,57,66,91.7,96.1,84.9,46.2,22.4,18.2,28.8,98.4,44.9,17.9,34.2,28.8,37.0,2008.0,67.1,280760008,John Pelphrey,Dennis Felton,2008,NaN,NaN
225,2008-03-16,conf_t,Georgia,SEC,Arkansas,SEC,N,W,66,57,104.8,78.4,98.4,44.9,17.9,34.2,28.8,84.9,46.2,22.4,18.2,28.8,96.6,2008.0,67.1,280760008,Dennis Felton,John Pelphrey,2008,NaN,NaN
226,2008-03-16,conf_t,Clemson,ACC,North Carolina,ACC,N,L,81,86,108.9,85.9,98.3,48.6,20.6,33.3,37.7,104.4,52.2,24.3,50.0,34.8,93.8,2008.0,82.4,280760153,Oliver Purnell,Roy Williams,2008,NaN,NaN
227,2008-03-16,conf_t,North Carolina,ACC,Clemson,ACC,N,W,86,81,116.8,88.4,104.4,52.2,24.3,50.0,34.8,98.3,48.6,20.6,33.3,37.7,96.1,2008.0,82.4,280760153,Roy Williams,Oliver Purnell,2008,NaN,NaN
228,2008-03-16,conf_t,Kansas,B12,Texas,B12,N,W,84,74,138.9,96.3,129.1,63.2,15.4,32.3,45.3,113.8,50.0,6.1,20.0,18.8,98.5,2008.0,65.1,280760251,Bill Self,Rick Barnes,2008,NaN,NaN


In [76]:
# Pre and During NCAA Tourney Split of Data, Include Rolling Averages For Team Performance (Values as of Date)

# Import Team Ratings By Day (2015-2019)
team_ratings = pd.read_csv('../data/cbbdata/team/team_ratings.csv').query('year != 2020')
team_ratings['date']= pd.to_datetime(team_ratings['date'])

# Split Daily Team Ranknings By Before/After Selection Sunday
ss_dict = {2024: "2024-3-17", 2023: "2023-3-12", 2022: "2022-3-13", 
           2021: "2021-3-14", 2019: "2019-3-17", 2018: "2018-3-11", 
           2017: "2017-3-12", 2016: "2016-3-13", 2015: "2015-3-15"}

# Create A Pre and During NCAA Tournament Day By Day Ratings
team_rating_pre_ncaa = pd.DataFrame(columns = team_ratings.columns)
team_rating_ncaa = pd.DataFrame(columns = team_ratings.columns)
for year, ss_date in ss_dict.items():
    team_rating_pre_ncaa = pd.concat([team_rating_pre_ncaa, team_ratings[(team_ratings['year'] == year) & (team_ratings['date'] < ss_date)]])    
    team_rating_ncaa = pd.concat([team_rating_ncaa, team_ratings[(team_ratings['year'] == year) & (team_ratings['date'] > ss_date)]])
    
# Look At Team Rating By Day and Calculate Rolling Adj Offensive Rank
team_rating_pre_ncaa['rolling_avg_adj_o_rk'] = team_rating_pre_ncaa.groupby(['team', 'year'])['adj_o'].transform(lambda x: x.rolling(window=3).mean())
team_rating_pre_ncaa[(team_rating_pre_ncaa['team'] == 'Kentucky') & (team_rating_pre_ncaa['year'] == 2015)].head()

In [298]:
selection_sunday_resume = pd.read_csv('../data/cbbdata/team/selection_sunday_resume.csv')
selection_sunday_resume[selection_sunday_resume['year'] == '2024'].head()

,team,year,net,resume,wab,elo,power,q1_w,q2_w,q3q4_l,games_above_500,score,seed,type,rd
1,Purdue,2024,3,1,2,3,3,12,20,0,25,99.96,???,???,???
31,Connecticut,2024,2,2,1,1,1.5,13,21,0,29,99.98,???,???,???
34,Houston,2024,1,3,3,2,1.5,16,19,0,27,99.97,???,???,???
53,Iowa St.,2024,6,4,4,4,5.5,10,16,0,21,99.86,???,???,???
75,North Carolina,2024,8,5,5,6,10,9,16,0,20,99.91,???,???,???
